In [ ]:
import os
os.getcwd()

In [ ]:
# import tensorflow as tf

In [ ]:
import os
from tensorboard import summary
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class BNNDCNet(tf.keras.Model):
    def __init__(self, input_shape=(224,224,3), num_disease_classes=3, num_species_classes=3):
        super(BNNDCNet, self).__init__(name = "")

        self.shape_input = input_shape
        self.num_disease_classes = num_disease_classes
        self.num_species_classes = num_species_classes
   
        self.neck = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2,2)),

            tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2,2)),

            tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2,2)),

            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(self.num_species_classes, activation='softmax'),

        ])

    def call(self, image_tensor, num_disease_classes=3, num_species_classes=3):
            layers = self.neck.layers
            output = image_tensor
            for layer in layers[:-2]:
                output = layer(output)
            conv_neck_output = output
            species_classification_output = self.neck(image_tensor) 


            return (species_classification_output, conv_neck_output)
        # self.body = tf.keras.Sequential([
        #     tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        #     tf.keras.layers.BatchNormalization(),
        #     tf.keras.layers.MaxPooling2D((2,2)),
        # ])

In [ ]:
# Compile the model.
# model = BNNDCNet()
# model.compile(loss={'binary_prediction_head': 'binary_crossentropy', 'disease_prediction_head': 'categorical_crossentropy'}, optimizer='adam', metrics=['accuracy'])

In [ ]:
from PIL import Image
img_path= r"E:\Corn_Wheat_Dataset\Wheat__yellow_rust\lolr(40).JPG"


In [ ]:
import numpy as np
def load_and_preprocess_image(image_path):
    # Load the image from the file path
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)  # Ensure it has 3 channels

    # Resize and crop the image to 224x224
    image = tf.image.resize(image, (256, 256))  # Resize to a larger size
    image = tf.image.random_crop(image, size=[224, 224, 3])  # Randomly crop to 224x224
    image = tf.image.random_flip_left_right(image)  # Randomly flip horizontally (optional)

    # Normalize the pixel values to the [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0
    return image

image_tensor = load_and_preprocess_image(img_path)
image_tensor = tf.expand_dims(image_tensor, axis=0)

# print(image_tensor)

In [ ]:
image_tensor

In [14]:
# Example usage:
input_shape = (224, 224, 3)
num_disease_classes = 3  # Assuming 3 disease classes
num_species_classes = 2  # Assuming 2 species classes
model = BNNDCNet(input_shape, num_disease_classes, num_species_classes)
# model.build_model()
# Wrap the image tensor in a batch dimension.



In [15]:
# layers = model.neck.layers
# output = image_tensor
# for layer in layers[:-2]:
#     output = layer(output)

output = model(image_tensor)

In [16]:
output

(<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.52965456, 0.47034544]], dtype=float32)>,
 <tf.Tensor: shape=(1, 28, 28, 256), dtype=float32, numpy=
 array([[[[0.05338345, 0.18323705, 0.0869627 , ..., 0.07791527,
           0.17302197, 0.06479341],
          [0.        , 0.19613664, 0.09701678, ..., 0.09431785,
           0.16671818, 0.06761944],
          [0.        , 0.224177  , 0.10090378, ..., 0.10112184,
           0.17394574, 0.07610101],
          ...,
          [0.        , 0.2014963 , 0.08990747, ..., 0.08474667,
           0.16778511, 0.06371424],
          [0.        , 0.18453677, 0.07925399, ..., 0.07735298,
           0.15292805, 0.05570308],
          [0.        , 0.18778133, 0.07682764, ..., 0.06389066,
           0.121874  , 0.06333181]],
 
         [[0.02337978, 0.19465412, 0.08082329, ..., 0.10106216,
           0.17664571, 0.04002831],
          [0.        , 0.20316008, 0.0923388 , ..., 0.11501164,
           0.16679513, 0.03785555],
          [0.        , 0